**[MDT-01]** 필요한 모듈을 임포트하고 난수의 시드를 설정한다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160703)
tf.set_random_seed(20160703)

**[MDT-02]** MNIST 데이터 세트를 준비한다.

In [2]:
mnist = input_data.read_data_sets("tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tenso

**[MDT-03]** 합성곱 필터가 한 계층인 CNN을 표현하는 클래스를 정의한다.

In [12]:
class SingleCNN:
    def __init__(self, num_filters, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_filters, num_units)
            self.prepare_session()

    def prepare_model(self, num_filters, num_units):
        num_units1 = 14*14*num_filters
        num_units2 = num_units
        
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
            x_image = tf.reshape(x, [-1,28,28,1])

        with tf.name_scope('convolution'):
            W_conv = tf.Variable(
                tf.truncated_normal([5,5,1,num_filters], stddev=0.1),
                name='conv-filter')
            h_conv = tf.nn.conv2d(
                x_image, W_conv, strides=[1,1,1,1], padding='SAME',
                name='filter-output')

        with tf.name_scope('pooling'):            
            h_pool =tf.nn.max_pool(h_conv, ksize=[1,2,2,1],
                                   strides=[1,2,2,1], padding='SAME',
                                   name='max-pool')
            h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters],
                                     name='pool-output')

        with tf.name_scope('fully-connected'):
            w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
            b2 = tf.Variable(tf.zeros([num_units2]))
            hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2,
                                 name='fc-output')

        with tf.name_scope('softmax'):
            w0 = tf.Variable(tf.zeros([num_units2, 10]))
            b0 = tf.Variable(tf.zeros([10]))
            p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0,
                              name='softmax-output')
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
            
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                              tf.float32), name='accuracy')
            
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.histogram("convolution_filters", W_conv)
        
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
        
    def prepare_session(self):
        sess = tf.InteractiveSession()
        sess.run(tf.initialize_all_variables())
        summary =  tf.summary.merge_all()
        writer = tf.summary.FileWriter("tmp/mnist_df_logs", sess.graph)
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

**[MDT-04]** 텐서보드용 데이터 출력 디렉토리를 삭제해서 초기화해둔다.

In [4]:
!rm -rf /tmp/mnist_df_logs

'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


**[MDT-05]** 파라미터 최적화를 4000회 반복한다. 테스트 세트에 대해 약 98%의 정답률을 얻을 수 있다.

In [13]:
cnn = SingleCNN(16, 1024)

i = 0
for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    cnn.sess.run(cnn.train_step, feed_dict={cnn.x:batch_xs, cnn.t:batch_ts})
    if i % 50 == 0:
        summary, loss_val, acc_val = cnn.sess.run(
            [cnn.summary, cnn.loss, cnn.accuracy],
            feed_dict={cnn.x:mnist.test.images, cnn.t:mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        cnn.writer.add_summary(summary, i)

c:\workspace\mytasks_ven\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Step: 50, Loss: 2934.805176, Accuracy: 0.912100
Step: 100, Loss: 2257.434082, Accuracy: 0.935400
Step: 150, Loss: 1858.536743, Accuracy: 0.950700
Step: 200, Loss: 1867.811035, Accuracy: 0.945500
Step: 250, Loss: 1498.115723, Accuracy: 0.958400
Step: 300, Loss: 1352.285889, Accuracy: 0.960300
Step: 350, Loss: 1263.737061, Accuracy: 0.962200
Step: 400, Loss: 1152.345215, Accuracy: 0.965000
Step: 450, Loss: 1200.433594, Accuracy: 0.963000
Step: 500, Loss: 1022.340454, Accuracy: 0.969100
Step: 550, Loss: 1130.758057, Accuracy: 0.967200
Step: 600, Loss: 965.390808, Accuracy: 0.972100
Step: 650, Loss: 873.780518, Accuracy: 0.973800
Step: 700, Loss: 951.643799, Accuracy: 0.972100
Step: 750, Loss: 905.574524, Accuracy: 0.973300
Step: 800, Loss: 922.026062, Accuracy: 0.971800
Step: 850, Loss: 791.417358, Accuracy: 0.976100
Step: 900, Loss: 841.286682, Accuracy: 0.973600
Step: 950, Loss: 802.439880, Accuracy: 0.975900
Step: 1000, Loss: 791.272766, Accuracy: 0.976300
Step: 1050, Loss: 874.285400,

tensorboard --host=127.0.0.1 --logdir=C:\workspace\myproject\mytasks\TensorflowDeeplearning-master\Chapter04\tmp\mnist_df_logs